In [1]:
from copy import deepcopy

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError

In [2]:
CONFIGS = {
    'data_path': '../data/',
    'model_path': '../model/',
    'model_name': 'using_dataset',

    'valid_start_index': 1704,
    'test_start_index': 1872,
    
    'batch_size': 64,
    'learning_rate': 1e-4,
    'epochs': 100,
    'es_patience': 10,
    
    'window_size': 7*24,
    'target_length': 3,
}

In [3]:
data_path = '../data/'

train_origin = pd.read_csv(data_path+'train.csv', encoding='cp949')

In [4]:
data = deepcopy(train_origin)

data.columns = [
    'num', 'date_time', 'target', 'temp', 'wind',
    'humid', 'rain', 'sun', 'non_elec_eq', 'sunlight_eq'
]

data = data.loc[data['num'] == 1, ['date_time', 'target']]

print(f'data.shape: {data.shape}')

data.shape: (2040, 2)


In [5]:
def mk_mean_std_dict(data):
    mean_std_dict = {
        col: {
            'mean': data[col].mean(),
            'std': data[col].std()
        } for col in data.columns
    }
    return mean_std_dict

In [6]:
scaling_cols = ['target']

mean_std_dict = mk_mean_std_dict(data[scaling_cols][:CONFIGS['valid_start_index']])
CONFIGS['mean_std_dict'] = mean_std_dict

In [7]:
def standard_scaling(data, mean_std_dict=None):
    if not mean_std_dict:
        mean_std_dict = mk_mean_std_dict(data)
    new_data = data.copy()
    for col in new_data.columns:
        new_data[col] -= mean_std_dict[col]['mean']
        new_data[col] /= mean_std_dict[col]['std']
    return new_data

In [8]:
data[scaling_cols] = standard_scaling(data[scaling_cols])

In [9]:
train = data.loc[:CONFIGS['valid_start_index'], 'target']
valid = data.loc[CONFIGS['valid_start_index']-CONFIGS['window_size']:CONFIGS['test_start_index'], 'target']
test = data.loc[CONFIGS['test_start_index']-CONFIGS['window_size']:, 'target']

In [10]:
def mk_dataset(data, shuffle=False):
    
    X = data[:-CONFIGS['target_length']]
    y = data[CONFIGS['window_size']:]
    
    X_ds = Dataset.from_tensor_slices(X)
    X_ds = X_ds.window(CONFIGS['window_size'], shift=1, drop_remainder=True)
    X_ds = X_ds.flat_map(lambda x: x).batch(CONFIGS['window_size'])
    
    y_ds = Dataset.from_tensor_slices(y)
    y_ds = y_ds.window(CONFIGS['target_length'], shift=1, drop_remainder=True)
    y_ds = y_ds.flat_map(lambda x: x).batch(CONFIGS['target_length'])
    
    ds = Dataset.zip((X_ds, y_ds))
    if shuffle:
        ds = ds.shuffle(512)
    ds = ds.batch(CONFIGS['batch_size']).cache().prefetch(2)
    
    return ds

In [11]:
train_ds = mk_dataset(train, shuffle=True)
valid_ds = mk_dataset(valid)
test_ds = mk_dataset(test)

In [12]:
def inversed_rmse(y_true, y_pred, mean, std):
    y_true = (y_true+mean)*std
    y_pred = (y_pred+mean)*std
    mse = tf.reduce_mean((y_true-y_pred)**2)
    return tf.sqrt(mse)

inversed_rmse_metric = lambda y_true, y_pred: inversed_rmse(y_true, y_pred, **CONFIGS['mean_std_dict']['target'])

In [13]:
def set_model(CONFIGS, model_name = None, print_summary=False):
    inputs = Input(batch_shape=(None, CONFIGS['window_size']), name='inputs')
    dense_0 = Dense(64, activation='relu', name='dense_0')(inputs)
    dense_1 = Dense(32, activation='relu', name='dense_1')(dense_0)
    outputs = Dense(CONFIGS['target_length'], name='outputs')(dense_1)
    
    if not model_name:
        model_name = CONFIGS['model_name']
    
    model = Model(
        inputs, outputs,
        name = model_name
    )
    
    optimizer = Adam(learning_rate=CONFIGS['learning_rate'])
    model.compile(
        loss = MeanSquaredError(),
        optimizer = optimizer,
        metrics=[inversed_rmse_metric],
    )
    
    if print_summary:
        model.summary()
    
    return model

In [14]:
model = set_model(CONFIGS, print_summary=True)

Model: "using_dataset"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 168)]             0         
_________________________________________________________________
dense_0 (Dense)              (None, 64)                10816     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
outputs (Dense)              (None, 3)                 99        
Total params: 12,995
Trainable params: 12,995
Non-trainable params: 0
_________________________________________________________________


In [15]:
def train_model(model, train_ds, valid_ds, CONFIGS):
    
    early_stop = EarlyStopping(
        patience=CONFIGS['es_patience']
    )
    save_best_only = ModelCheckpoint(
        filepath = f'{CONFIGS["model_path"]}{CONFIGS["model_name"]}.h5',
        monitor = 'val_loss',
        save_best_only = True,
        save_weights_only = True
    )
    
    history = model.fit(
        train_ds,
        batch_size = CONFIGS['batch_size'],
        epochs = CONFIGS['epochs'],
        validation_data = valid_ds,
        callbacks = [
            early_stop,
            save_best_only,
        ]
    )
    
    return history

In [16]:
history = train_model(model, train_ds, valid_ds, CONFIGS)

Epoch 1/100
24/24 [==============================] - 1s 8ms/step - loss: 0.5110 - <lambda>: 89.7635 - val_loss: 0.6664 - val_<lambda>: 103.8948
Epoch 2/100
24/24 [==============================] - 0s 2ms/step - loss: 0.3783 - <lambda>: 77.1876 - val_loss: 0.4935 - val_<lambda>: 86.5560
Epoch 3/100
24/24 [==============================] - 0s 2ms/step - loss: 0.3056 - <lambda>: 69.3042 - val_loss: 0.4103 - val_<lambda>: 76.7236
Epoch 4/100
24/24 [==============================] - 0s 2ms/step - loss: 0.2592 - <lambda>: 63.7424 - val_loss: 0.3589 - val_<lambda>: 70.6262
Epoch 5/100
24/24 [==============================] - 0s 2ms/step - loss: 0.2280 - <lambda>: 59.7074 - val_loss: 0.3188 - val_<lambda>: 66.0296
Epoch 6/100
24/24 [==============================] - 0s 2ms/step - loss: 0.2057 - <lambda>: 56.6643 - val_loss: 0.2850 - val_<lambda>: 62.1642
Epoch 7/100
24/24 [==============================] - 0s 2ms/step - loss: 0.1891 - <lambda>: 54.3020 - val_loss: 0.2565 - val_<lambda>: 58.882

Epoch 58/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0604 - <lambda>: 30.0124 - val_loss: 0.1161 - val_<lambda>: 40.5032
Epoch 59/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0598 - <lambda>: 29.8541 - val_loss: 0.1156 - val_<lambda>: 40.4321
Epoch 60/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0593 - <lambda>: 29.7107 - val_loss: 0.1149 - val_<lambda>: 40.3417
Epoch 61/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0588 - <lambda>: 29.5633 - val_loss: 0.1146 - val_<lambda>: 40.2893
Epoch 62/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0582 - <lambda>: 29.4190 - val_loss: 0.1141 - val_<lambda>: 40.2215
Epoch 63/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0577 - <lambda>: 29.2759 - val_loss: 0.1141 - val_<lambda>: 40.2367
Epoch 64/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0572 - <lambda>: 29.1372 - val_loss: 0.1137 - val_<lambda>: 

In [17]:
best_model = set_model(CONFIGS, model_name='best_'+CONFIGS['model_name'])
best_model.load_weights(f'{CONFIGS["model_path"]}{CONFIGS["model_name"]}.h5')

In [18]:
y_train_pred = best_model.predict(train_ds)
y_valid_pred = best_model.predict(valid_ds)
y_test_pred = best_model.predict(test_ds)

In [19]:
train_loss, train_rmse = best_model.evaluate(train_ds, verbose=0)
valid_loss, valid_rmse = best_model.evaluate(valid_ds, verbose=0)
test_loss, test_rmse = best_model.evaluate(test_ds, verbose=0)

print(f'train_loss: {train_loss:.6f}\ttrain_rmse: {train_rmse:.6f}')
print(f'valid_loss: {valid_loss:.6f}\tvalid_rmse: {valid_rmse:.6f}')
print(f'test_loss: {test_loss:.6f}\ttest_rmse: {test_rmse:.6f}')

train_loss: 0.043031	train_rmse: 24.979258
valid_loss: 0.104636	valid_rmse: 39.018162
test_loss: 0.128503	test_rmse: 45.357426
